In [2]:
import boto3
import pandas as pd
import os.path

In [25]:
s3 = boto3.resource('s3')
client = boto3.client("s3")

bucket = "ehrlich-arb-data"
dateid = "20171206"

buck = s3.Bucket(bucket)

files = []
for o in buck.objects.filter(Prefix=dateid).all():
    files.append(o.key)

if os.path.isdir(dateid) == False:
    os.mkdir(dateid)    
exists = []
for f in files:
    exists.append(os.path.isfile(f))

In [29]:
for f in files:
    if os.path.isfile(f) == False:
        print(f)
        client.download_file(bucket, f, f)

In [33]:
l = []
for (i, f) in enumerate(files):
    df = pd.read_csv(f, header=None)
    l.append(df)
    df.head()
#data = pd.concat(l)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
data.head()

,0.0004,1.5,124,1512531000848,1512576004367,3.25,COINGI,PPCBTC,PPCUSD
0,NaN,99.947,NaN,NaN,1.512576e+12,3.33,COINGI,NaN,PPCUSD
1,NaN,12.500,NaN,NaN,1.512576e+12,3.43,COINGI,NaN,PPCUSD
2,NaN,35.400,NaN,NaN,1.512576e+12,3.56,COINGI,NaN,PPCUSD
3,NaN,2.667,NaN,NaN,1.512576e+12,3.84,COINGI,NaN,PPCUSD
4,NaN,12.500,NaN,NaN,1.512576e+12,4.54,COINGI,NaN,PPCUSD


In [16]:
df1 = pd.read_csv("20171206/ls.s3.0c288acf-b30c-4448-a816-f5c863b74be4.2017-12-06T16.00.part32.txt.gz", header = None)
df1.head()

,0,1,2,3,4
0,1512576004367,COINGI,PPCUSD,3.25,1.500
1,1512576004367,COINGI,PPCUSD,3.33,99.947
2,1512576004367,COINGI,PPCUSD,3.43,12.500
3,1512576004367,COINGI,PPCUSD,3.56,35.400
4,1512576004367,COINGI,PPCUSD,3.84,2.667


In [17]:
df2 = pd.read_csv("20171206/ls.s3.1511ca11-bb4a-4dbe-a60b-fde55b70dc11.2017-12-06T03.30.part7.txt.gz", header = None)
df2.head()

,0,1,2,3,4
0,1512531000848,COINGI,PPCBTC,0.00040,124.00
1,1512531000848,COINGI,PPCBTC,0.00043,39.10
2,1512531000848,COINGI,PPCBTC,0.00046,43.50
3,1512531000848,COINGI,PPCBTC,0.00049,369.56
4,1512531000848,COINGI,PPCBTC,0.00051,321.00


In [18]:
df3 = pd.concat([df1, df2])
df3.head()

,0,1,2,3,4
0,1512576004367,COINGI,PPCUSD,3.25,1.500
1,1512576004367,COINGI,PPCUSD,3.33,99.947
2,1512576004367,COINGI,PPCUSD,3.43,12.500
3,1512576004367,COINGI,PPCUSD,3.56,35.400
4,1512576004367,COINGI,PPCUSD,3.84,2.667


In [30]:
data[0].head()

0    2017-12-04T01:21:15.271Z Andrews-MBP.gateway.s...
1    2017-12-04T01:21:15.271Z Andrews-MBP.gateway.s...
2    2017-12-04T01:21:15.271Z Andrews-MBP.gateway.s...
3    2017-12-04T01:21:15.271Z Andrews-MBP.gateway.s...
4    2017-12-04T01:21:15.271Z Andrews-MBP.gateway.s...
Name: 0, dtype: object